In [27]:
%pip install cohere


[notice] A new release of pip is available: 23.0.1 -> 23.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [11]:
%pip install guardrails-ai pydantic typing openai rich

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import guardrails as gd

In [9]:
from pydantic import BaseModel, Field
from guardrails.validators import ValidLength, TwoWords, ValidRange
from guardrails.datatypes import Date
from typing import List

prompt = """
Generate a dataset of fake user orders. Each row of the dataset should be valid. The format should not be a list, it should be a JSON object.
${gr.complete_json_suffix}

an example of output may look like this:
{
	"user_orders": [{                                                                                                   │ │
        "user_id": 1,
        "user_name": "John Mcdonald",
    	"num_orders": 6
    }]
}

"""

class Order(BaseModel):
	user_id: int = Field(description="The user's id.", validators=[("1-indexed", "noop")])
	user_name: str = Field(
		description="The user's first name and last name",
		validators=[TwoWords()]
	)

	num_orders: int = Field(
		description="The number of orders the user has placed",
		validators=[ValidRange(0, 50)]
	)
	


class Orders(BaseModel):
	user_orders: List[Order] = Field(
		description="Generate a list of users and how many orders they have placed in the past.",
		validators=[ValidLength(10, 10, on_fail="noop")]
	)

In [2]:
import cohere

co = cohere.Client(api_key='<Cohere_API_KEY>')

In [10]:
guard = gd.Guard.from_pydantic(output_class=Orders, prompt=prompt)

In [11]:
raw_llm_response, validated_response = guard(
	co.generate,
	model="command",
	max_tokens=1024,
	temperature=0.3
)

In [12]:
from rich import print

guard.guard_state.most_recent_call.tree

Logs
└── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
    │ ╭──────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────╮ │
    │ │                                                                                                         │ │
    │ │ Generate a dataset of fake user orders. Each row of the dataset should be valid. The format should not  │ │
    │ │ be a list, it should be a JSON object.                                                                  │ │
    │ │                                                                                                         │ │
    │ │ Given below is XML that describes the information to extract from this document and the tags to extract │ │
    │ │ it into.                                                                                                │ │
    │ │                                                                                                         │ │
    │ │ <output>                                                                                                │ │
    │ │     <list name="user_orders" format="length: min=10 max=10" description="Generate a list of users and   │ │
    │ │ how many orders they have placed in the past.">                                                         │ │
    │ │         <object>                                                                                        │ │
    │ │             <integer name="user_id" format="1-indexed" description="The user's id."/>                   │ │
    │ │             <string name="user_name" format="two-words" description="The user's first name and last     │ │
    │ │ name"/>                                                                                                 │ │
    │ │             <integer name="num_orders" format="valid-range: min=0 max=50" description="The number of    │ │
    │ │ orders the user has placed"/>                                                                           │ │
    │ │         </object>                                                                                       │ │
    │ │     </list>                                                                                             │ │
    │ │ </output>                                                                                               │ │
    │ │                                                                                                         │ │
    │ │                                                                                                         │ │
    │ │ ONLY return a valid JSON object (no other text is necessary), where the key of the field in JSON is the │ │
    │ │ `name` attribute of the corresponding XML, and the value is of the type specified by the corresponding  │ │
    │ │ XML's tag. The JSON MUST conform to the XML format, including any types and format requests e.g.        │ │
    │ │ requests for lists, objects and specific types. Be correct and concise. If you are unsure anywhere,     │ │
    │ │ enter `null`.                                                                                           │ │
    │ │                                                                                                         │ │
    │ │ Here are examples of simple (XML, JSON) pairs that show the expected behavior:                          │ │
    │ │ - `<string name='foo' format='two-words lower-case' />` => `{'foo': 'example one'}`                     │ │
    │ │ - `<list name='bar'><string format='upper-case' /></list>` => `{"bar": ['STRING ONE', 'STRING TWO',     │ │
    │ │ etc.]}`                                                                                                 │ │
    │ │ - `<object name='baz'><string name="foo" format="capitalize two-words" /><integer name="index"          │ │
    │ │ format="1-indexed" /></object>` => `{'baz':